In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import matplotlib.pyplot as plt
import os
import json
import gzip

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from zipfile import ZipFile
from pathlib import Path
from wordcloud import WordCloud, STOPWORDS
from pandas import DataFrame 
from sklearn.feature_extraction.text import CountVectorizer 
from urllib.request import urlopen
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
! wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_10.json.gz

--2020-07-26 14:30:35--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_10.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129676119 (124M) [application/x-gzip]
Saving to: ‘reviews_Electronics_10.json.gz’

reviews_Electronics 100%[===================>] 123.67M  13.3MB/s    in 7.4s    

2020-07-26 14:30:43 (16.6 MB/s) - ‘reviews_Electronics_10.json.gz’ saved [129676119/129676119]



In [ ]:
data = []
with gzip.open('reviews_Electronics_10.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
print(len(data))
print(data[0])

347393
{'reviewerID': 'A2IDCSC6NVONIZ', 'asin': '0972683275', 'reviewerName': '2Cents!', 'helpful': [1, 1], 'reviewText': 'This mount is just what I needed.  It is strong and sturdy.  It folds almost flat to enable the television to sit square and flat against the wall if desired and extends and rotates for angle turns.  A perfect fit as long as you match the correct visa pattern for your television.', 'overall': 5.0, 'summary': 'Perfect', 'unixReviewTime': 1367280000, 'reviewTime': '04 30, 2013'}


In [ ]:
regex = re.compile('[^a-z]+')
def cleanup(sentence):
    sentence = sentence.lower()
    sentence = regex.sub(' ', sentence).strip()
    sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

In [ ]:
df = pd.DataFrame.from_dict(data)
df["reviewText"] = df["reviewText"].apply(cleanup) #df["summary"].apply(cleanup)
df[:5]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IDCSC6NVONIZ,0972683275,2Cents!,"[1, 1]",this mount is just what i needed it is strong ...,5.0,Perfect,1367280000,"04 30, 2013"
1,A3BMUBUC1N77U8,0972683275,ahoffoss,"[0, 0]",this mount works really well once you get it u...,4.0,"Pretty simple, but definitely good!",1385164800,"11 23, 2013"
2,AQBLWW13U66XD,0972683275,"Benjamin Belanger ""v dbl u""","[0, 0]",i bought this for a tv for my son i mounted th...,5.0,High Quality/Low Price,1375574400,"08 4, 2013"
3,A3IIGCFLKVFW8M,0972683275,"Brian M. Kaplan ""Brian M. Kaplan""","[0, 0]",works great and is so much cheaper than the mo...,5.0,Holds a lot,1393459200,"02 27, 2014"
4,A3VKO21KYDJQ2W,0972683275,C. Aaland,"[0, 0]",for the price you can t beat it mine didn t co...,3.0,Great for the price.,1310428800,"07 12, 2011"


ProductBased Collbrative Filtering


In [ ]:
count = df.groupby("asin", as_index=False).count()
mean = df.groupby("asin", as_index=False).mean()

In [ ]:
df_temp = pd.merge(df, count, how='right', on=['asin'])
df_temp = pd.merge(df_temp, mean, how='right', on=['asin'])

In [ ]:
df_temp[:5]

,reviewerID_x,asin,reviewerName_x,helpful_x,reviewText_x,overall_x,summary_x,unixReviewTime_x,reviewTime_x,reviewerID_y,reviewerName_y,helpful_y,reviewText_y,overall_y,summary_y,unixReviewTime_y,reviewTime_y,overall,unixReviewTime
0,A2IDCSC6NVONIZ,0972683275,2Cents!,"[1, 1]",this mount is just what i needed it is strong ...,5.0,Perfect,1367280000,"04 30, 2013",50,50,50,50,50,50,50,50,4.46,1.351897e+09
1,A3BMUBUC1N77U8,0972683275,ahoffoss,"[0, 0]",this mount works really well once you get it u...,4.0,"Pretty simple, but definitely good!",1385164800,"11 23, 2013",50,50,50,50,50,50,50,50,4.46,1.351897e+09
2,AQBLWW13U66XD,0972683275,"Benjamin Belanger ""v dbl u""","[0, 0]",i bought this for a tv for my son i mounted th...,5.0,High Quality/Low Price,1375574400,"08 4, 2013",50,50,50,50,50,50,50,50,4.46,1.351897e+09
3,A3IIGCFLKVFW8M,0972683275,"Brian M. Kaplan ""Brian M. Kaplan""","[0, 0]",works great and is so much cheaper than the mo...,5.0,Holds a lot,1393459200,"02 27, 2014",50,50,50,50,50,50,50,50,4.46,1.351897e+09
4,A3VKO21KYDJQ2W,0972683275,C. Aaland,"[0, 0]",for the price you can t beat it mine didn t co...,3.0,Great for the price.,1310428800,"07 12, 2011",50,50,50,50,50,50,50,50,4.46,1.351897e+09


In [ ]:
df_temp["Count"] = df_temp["reviewerID_y"]
df_temp["Score"] = df_temp["overall"]
df_temp["Summary"] = df_temp["reviewText_x"]
df_temp["ProductID"] = df_temp["asin"]

In [ ]:
df_temp = df_temp[['ProductID','Summary','Score',"Count"]]

In [ ]:
df_temp = df_temp.sort_values(by=['Count'], ascending=False)

In [ ]:
df["ProductID"] = df["asin"]
df1 = df.groupby("ProductID", as_index=False).mean()
combine_summary = df_temp.groupby("ProductID")["Summary"].apply(list)
combine_summary = pd.DataFrame(combine_summary)
combine_summary.to_csv("combine_summary.csv")

In [ ]:
df1.head()

,ProductID,overall,unixReviewTime
0,0972683275,4.460000,1.351897e+09
1,1400501466,3.666667,1.342145e+09
2,1400532620,4.000000,1.297987e+09
3,1400532655,3.741935,1.307714e+09
4,140053271X,3.736842,1.337281e+09


In [ ]:
df2 = pd.read_csv("combine_summary.csv")
df2 = pd.merge(df2, df_temp, on="ProductID", how='inner')
df2 = df2[['ProductID','Summary_x','Score']]

In [ ]:
df2.head()

,ProductID,Summary_x,Score
0,0972683275,['this mount works really well once you get it...,4.46
1,0972683275,['this mount works really well once you get it...,4.46
2,0972683275,['this mount works really well once you get it...,4.46
3,0972683275,['this mount works really well once you get it...,4.46
4,0972683275,['this mount works really well once you get it...,4.46


In [ ]:
df2 = df2.drop_duplicates(['Score'], keep='last')
df2 = df2.reset_index()

In [ ]:
df2.head()

,index,ProductID,Summary_x,Score
0,49,0972683275,['this mount works really well once you get it...,4.460000
1,102,1400532655,['i m sure a lot of other potential e book wan...,3.741935
2,1002,B00001WRSJ,['bought this item for use at the office at my...,4.579439
3,1134,B00002EQCW,['i bought this little switch to run either my...,4.578125
4,1376,B00004SABB,['i returned this i could not adjust it all ma...,3.879310


In [ ]:
from pandas import DataFrame 
docs = df2["Summary_x"] 
vect = CountVectorizer(max_features = 100, stop_words='english') 
X = vect.fit_transform(docs) 
df2["Summary"] = df2["Summary_x"]  
df3 = DataFrame(X.A, columns=vect.get_feature_names())
df3 = df3.astype(int)

In [ ]:
df3

,able,amazon,apple,battery,best,better,bit,bought,box,buy,cable,cables,camera,card,case,computer,device,devices,did,didn,does,doesn,don,drive,easy,far,fast,fine,gb,going,good,got,great,hard,having,high,home,ipad,just,keyboard,...,phone,plug,power,pretty,price,problem,product,quality,really,recommend,right,router,say,screen,set,size,small,software,sound,speed,sure,tablet,thing,think,time,tv,unit,usb,use,used,using,ve,video,want,way,windows,wireless,work,worked,works
0,1,1,0,0,5,2,3,7,2,8,1,2,0,0,0,0,1,0,0,4,6,2,5,0,19,1,0,3,0,3,5,1,14,3,1,3,4,0,10,0,...,0,0,0,2,14,2,2,8,7,6,4,0,2,2,2,4,1,0,0,0,3,0,1,4,3,68,5,0,8,4,5,2,0,3,3,0,0,9,6,11
1,10,18,1,16,8,17,4,16,9,22,2,0,2,12,3,1,42,2,10,3,18,12,20,1,16,11,5,6,7,4,20,7,28,2,3,3,11,24,28,4,...,7,2,4,10,14,4,3,2,14,5,4,0,4,49,4,6,8,2,2,5,7,29,8,6,17,2,1,4,30,12,11,13,6,15,17,2,5,15,3,10
2,10,9,0,3,30,34,13,30,7,23,4,0,5,0,5,10,5,2,18,7,8,8,47,1,3,9,0,5,0,10,84,11,52,9,5,35,7,0,53,2,...,7,11,2,18,36,14,6,62,32,17,15,0,14,0,33,15,12,0,195,0,7,0,12,17,31,6,1,0,40,26,23,58,5,11,14,0,1,12,1,2
3,2,2,1,0,0,5,1,6,16,7,17,13,1,0,12,8,8,16,2,5,9,4,4,1,7,2,10,10,0,4,14,6,18,3,1,1,11,0,26,0,...,3,26,29,1,14,7,14,8,2,4,2,30,8,0,4,1,14,2,0,7,7,0,0,0,9,14,11,1,12,9,9,9,2,7,3,0,9,22,9,22
4,4,5,0,0,2,14,6,13,2,7,0,0,2,0,13,1,0,1,7,5,0,2,14,0,12,7,2,5,0,2,30,12,30,3,11,4,1,0,33,0,...,0,0,7,6,30,12,12,7,10,6,13,0,6,0,16,14,25,0,0,1,7,0,9,4,13,0,0,0,31,11,7,5,0,11,5,0,1,11,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,3,0,0,2,0,6,3,0,0,1,6,1,0,0,4,1,7,3,0,0,1,0,4,0,2,0,0,0,0,0,10,0,3,5,1,3,0,1,7,0,...,12,3,5,7,1,1,2,6,5,0,2,0,1,0,1,2,4,0,15,0,2,1,1,1,1,0,6,8,4,3,3,2,0,4,0,0,1,1,1,0
1888,3,0,0,1,0,2,3,0,4,0,13,0,6,74,3,10,5,6,4,1,3,2,8,2,4,5,4,1,4,1,4,2,7,0,0,2,1,0,9,0,...,1,9,4,1,0,2,2,1,7,3,2,0,1,0,0,2,4,0,2,4,1,0,3,1,6,0,17,36,8,3,7,1,0,2,5,4,0,3,2,3
1889,4,0,0,0,0,1,1,0,0,1,3,1,2,2,2,7,10,17,3,0,2,5,3,3,2,1,2,1,0,1,3,1,11,3,0,2,0,0,9,5,...,5,7,12,0,3,1,5,1,3,2,2,0,1,1,1,7,11,0,0,6,2,4,4,1,1,0,4,64,5,5,8,3,0,2,1,7,6,4,2,3
1890,3,1,9,9,3,13,4,0,3,0,14,0,0,0,2,4,47,12,0,2,9,4,8,0,9,3,1,0,0,2,7,4,3,0,3,0,15,10,8,0,...,13,11,23,1,1,4,12,12,2,3,2,1,2,0,6,1,5,0,17,1,3,5,7,2,4,1,8,28,35,10,11,3,2,9,6,9,24,4,3,15


In [ ]:
X = np.array(df3)
tpercent = 0.9
tsize = int(np.floor(tpercent * len(df3)))
df3_train = X[:tsize]
df3_test = X[tsize:]

lentrain = len(df3_train)
lentest = len(df3_test)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(df3_train)

distances, indices = nbrs.kneighbors(df3_train)

In [ ]:
for i in range(lentest):
    a = nbrs.kneighbors([df3_test[i]])
    related_product_list = a[1]
    
    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Product : ", df2["ProductID"][lentrain + i] ," Score : ",df2["Score"][lentrain + i])
    print ("Similar Product 1: ", df2["ProductID"][first_related_product] ," Score : ",df2["Score"][first_related_product])
    print ("Similar Product 2: ", df2["ProductID"][second_related_product] ," Score : ",df2["Score"][second_related_product])
    print ("---------------------------------------------------------------------------------------------------------")

Product :  B00FNPD1VW  Score :  4.6075949367088604
Similar Product 1:  B001QTXL82  Score :  4.673913043478261
Similar Product 2:  B005JN9310  Score :  4.731182795698925
---------------------------------------------------------------------------------------------------------
Product :  B00FO12XY6  Score :  4.64
Similar Product 1:  B00AWX6EYQ  Score :  4.3478260869565215
Similar Product 2:  B0076R7F1M  Score :  3.85
---------------------------------------------------------------------------------------------------------
Product :  B00FONBG1A  Score :  4.052631578947368
Similar Product 1:  B005RG99OU  Score :  4.621621621621622
Similar Product 2:  B00ATNL9AS  Score :  2.8181818181818183
---------------------------------------------------------------------------------------------------------
Product :  B00FPG37R2  Score :  3.3461538461538463
Similar Product 1:  B00DSJA3G6  Score :  2.4074074074074074
Similar Product 2:  B00EVIBN8U  Score :  4.55
--------------------------------------------

In [ ]:
df3_train_target = df2["Score"][:lentrain]
df3_test_target = df2["Score"][lentrain:lentrain+lentest]
df3_train_target = df3_train_target.astype(int)
df3_test_target = df3_test_target.astype(int)

n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(df3_train, df3_train_target)
knnpreds_test = knnclf.predict(df3_test)
print (knnpreds_test)

print(classification_report(df3_test_target, knnpreds_test))
print(accuracy_score(df3_test_target, knnpreds_test))

[4 4 4 2 4 4 3 4 3 3 3 3 4 4 3 4 4 4 3 4 4 4 4 4 4 3 3 4 4 3 3 3 3 3 2 3 3
 4 3 3 4 3 4 4 4 4 3 4 4 3 3 4 4 3 4 4 3 4 4 4 2 4 2 3 4 3 4 4 3 4 4 4 3 3
 3 4 4 4 4 4 4 4 4 4 3 4 3 3 4 4 2 3 3 4 3 3 3 4 3 4 3 4 4 4 3 4 4 3 4 3 3
 3 4 2 3 4 4 4 3 2 4 3 4 4 4 3 2 4 4 4 4 4 4 4 4 3 4 2 4 3 3 3 4 2 4 4 4 4
 4 3 4 4 4 4 4 4 2 4 4 4 2 4 3 3 3 3 3 3 4 4 4 4 4 3 4 2 4 3 3 3 3 4 3 4 3
 2 4 3 2 4]
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.37      0.52      0.43        48
           4       0.80      0.62      0.70       139
           5       0.00      0.00      0.00         1

    accuracy                           0.58       190
   macro avg       0.29      0.28      0.28       190
weighted avg       0.68      0.58      0.62       190

0.5842105263157895


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


UserBased Collabrative Filtering


In [ ]:
count = df.groupby("reviewerID", as_index=False).count()
mean = df.groupby("reviewerID", as_index=False).mean()

udf = pd.merge(df, count, how='right', on=["reviewerID"])
udf = pd.merge(udf, mean, how='right', on=["reviewerID"])
udf.head()

,reviewerID,asin_x,reviewerName_x,helpful_x,reviewText_x,overall_x,summary_x,unixReviewTime_x,reviewTime_x,ProductID_x,asin_y,reviewerName_y,helpful_y,reviewText_y,overall_y,summary_y,unixReviewTime_y,reviewTime_y,ProductID_y,overall,unixReviewTime
0,A2IDCSC6NVONIZ,0972683275,2Cents!,"[1, 1]",this mount is just what i needed it is strong ...,5.0,Perfect,1367280000,"04 30, 2013",0972683275,17,17,17,17,17,17,17,17,17,4.764706,1.357756e+09
1,A2IDCSC6NVONIZ,B00005T3N3,2Cents!,"[0, 0]",these stands are great they work with just abo...,5.0,Nice,1367193600,"04 29, 2013",B00005T3N3,17,17,17,17,17,17,17,17,17,4.764706,1.357756e+09
2,A2IDCSC6NVONIZ,B00005V51P,2Cents!,"[0, 0]",it s a coupler what else can you say about it ...,5.0,It Works,1402185600,"06 8, 2014",B00005V51P,17,17,17,17,17,17,17,17,17,4.764706,1.357756e+09
3,A2IDCSC6NVONIZ,B00007IFED,2Cents!,"[0, 0]",bought this for my microsost surface tablet wo...,5.0,Perfect,1404777600,"07 8, 2014",B00007IFED,17,17,17,17,17,17,17,17,17,4.764706,1.357756e+09
4,A2IDCSC6NVONIZ,B000V0ETA6,2Cents!,"[0, 0]",don t make the mistake of thinking you will ev...,5.0,The price is right,1367280000,"04 30, 2013",B000V0ETA6,17,17,17,17,17,17,17,17,17,4.764706,1.357756e+09


In [ ]:
udf["Count"] = udf["asin_y"]
udf["Score"] = udf["overall"]
udf["Summary"] = udf["reviewText_x"]
udf["UserId"] = udf["reviewerID"]
udf = udf[["UserId",'Summary','Score',"Count"]]

In [ ]:
udf[:5]

,UserId,Summary,Score,Count
0,A2IDCSC6NVONIZ,this mount is just what i needed it is strong ...,4.764706,17
1,A2IDCSC6NVONIZ,these stands are great they work with just abo...,4.764706,17
2,A2IDCSC6NVONIZ,it s a coupler what else can you say about it ...,4.764706,17
3,A2IDCSC6NVONIZ,bought this for my microsost surface tablet wo...,4.764706,17
4,A2IDCSC6NVONIZ,don t make the mistake of thinking you will ev...,4.764706,17


In [ ]:
udf = udf.sort_values(by=['Count'], ascending=False)

In [ ]:
udf1 = df.groupby("reviewerID", as_index=False).mean()
udf1["UserId"] = udf1["reviewerID"]
combine_summary = udf.groupby("UserId")["Summary"].apply(list)
combine_summary = pd.DataFrame(combine_summary)
combine_summary.to_csv("combine_summary.csv")

In [ ]:
udf2 = pd.read_csv("combine_summary.csv")
udf2 = pd.merge(udf2, udf1, on="UserId", how='inner')
udf2 = udf2[['UserId','Summary','overall']]

In [ ]:
udf2.head()

,UserId,Summary,overall
0,A0251761JI35FM4C8VK2,['lots of space love this hdd this thing is aw...,5.000000
1,A1002O7Q4BOWMM,['a bit pricy for what they are but it works t...,4.800000
2,A1004703RC79J9,['i received this through the vine program and...,4.250000
3,A10063PJ5C9WQQ,['this case and keyboard is an excellent addit...,4.300000
4,A100NGGXRQF0AQ,['i was interested in using this router to rep...,3.846154


In [ ]:
udf2 = udf2.drop_duplicates(['overall'], keep='last')
udf2 = udf2.reset_index()

In [ ]:
docs = udf2["Summary"] 
vect = CountVectorizer(max_features = 100, stop_words='english') 
X = vect.fit_transform(docs) 
#print(DataFrame(X.A, columns=vect.get_feature_names()).to_string()) 
udf3 = DataFrame(X.A, columns=vect.get_feature_names())
udf3 = udf3.astype(int)

In [ ]:
kkk  = df.drop_duplicates(['reviewText'], keep='last')
kkk = kkk.reset_index()

udf3.to_csv("udf3.csv")

In [ ]:
kkk.head()

,index,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,ProductID
0,0,A2IDCSC6NVONIZ,0972683275,2Cents!,"[1, 1]",this mount is just what i needed it is strong ...,5.0,Perfect,1367280000,"04 30, 2013",0972683275
1,1,A3BMUBUC1N77U8,0972683275,ahoffoss,"[0, 0]",this mount works really well once you get it u...,4.0,"Pretty simple, but definitely good!",1385164800,"11 23, 2013",0972683275
2,2,AQBLWW13U66XD,0972683275,"Benjamin Belanger ""v dbl u""","[0, 0]",i bought this for a tv for my son i mounted th...,5.0,High Quality/Low Price,1375574400,"08 4, 2013",0972683275
3,3,A3IIGCFLKVFW8M,0972683275,"Brian M. Kaplan ""Brian M. Kaplan""","[0, 0]",works great and is so much cheaper than the mo...,5.0,Holds a lot,1393459200,"02 27, 2014",0972683275
4,4,A3VKO21KYDJQ2W,0972683275,C. Aaland,"[0, 0]",for the price you can t beat it mine didn t co...,3.0,Great for the price.,1310428800,"07 12, 2011",0972683275


In [ ]:
X = np.array(udf3)

tpercent = 0.95
tsize = int(np.floor(tpercent * len(udf3)))
udf3_train = X[:tsize]
udf3_test = X[tsize:]

lentrain = len(udf3_train)
lentest = len(udf3_test)
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(udf3_train)

distances, indices = nbrs.kneighbors(udf3_train)

In [ ]:
for i in range(lentest):
    a = nbrs.kneighbors([udf3_test[i]])
    related_product_list = a[1]
    
    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Usr : ", udf2["UserId"][lentrain + i])
    print ("The first similar user : ", udf2["UserId"][first_related_product], ".") 
    print ("User might like products:")
    for i in range(295743):
        if (kkk["reviewerID"][i] == udf2["UserId"][first_related_product]) & (kkk["overall"][i] == 5):
            aaa= kkk["asin"][i]
            print (aaa),
    print ("--------------------------------------------------------------------")

Usr :  AZ9QAIBGTLTLB
The first similar user :  AU1EU8PNIOMXE .
User might like products:
B000N99BBC
B002ONCC6G
B005003WAA
B006JH8T3S
B0093LINVK
--------------------------------------------------------------------
Usr :  AZA595ZPIG240
The first similar user :  ALNE7Q71JFC0F .
User might like products:
B00006I5J7
B00007E89K
B0000BZL5A
B000BTCJNG
B000XB9GV0
B001ENW61I
B001FA09ZA
B001QGT1CA
B0037KM0X0
B004WDIV8E
B005GGTTZ0
B0076AYNXM
B008EQZ25K
--------------------------------------------------------------------
Usr :  AZAC8O310IK4E
The first similar user :  A38NHXL5257E3B .
User might like products:
B00004ZCA5
B0006I09LQ
B000I963FE
B000WL6YY8
B00109Y2DQ
B0012GQZZU
B001AYCNIW
B001D60LG8
B001FWKA7A
B0028Y4DCC
B002LITI8E
B002S53LJ2
B003EO1H7E
B003IRR0YQ
B003SGCO3E
B0041Q38N0
B00466X9SY
B004COCMRO
B004GUT1QO
B004J3ZV62
B004Q3R9AQ
B004T9RR6I
B004WYA852
B004XC6GJ0
B004ZMV2GA
B0052RM8T2
B0053NBLFW
B0059M4G9K
B005FYNSPK
B0067SF63A
B00698ZUHK
B006OBGEHW
B007KZQM8W
B007X3VC7W
B0081FLFQE
B00829THK0


In [ ]:
udf3_train_target = udf2["overall"][:lentrain]
udf3_test_target = udf2["overall"][lentrain:lentrain+lentest]
udf3_train_target = udf3_train_target.astype(int)
udf3_test_target = udf3_test_target.astype(int)

n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(udf3_train, udf3_train_target)
knnpreds_test = knnclf.predict(udf3_test)
print("Prediction for TestSet of Users: ", knnpreds_test)
print(classification_report(udf3_test_target, knnpreds_test))
print(accuracy_score(udf3_test_target, knnpreds_test))

Prediction for TestSet of Users:  [3 4 4 3 3 3 4 3 4 4 2 4 4 4 3 4 2 1 4 4 3 3 2 3 4 2 3 3 4 4 4 4 3 3 2 4 2
 2 2 3 3 3 2 4 3 4 2 4 4 4 3 3 4 3 4 3 3 4 4 2 4 2 4 4 3 4 3]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         3
           3       0.29      0.44      0.35        16
           4       0.80      0.51      0.62        47
           5       0.00      0.00      0.00         1

    accuracy                           0.46        67
   macro avg       0.22      0.19      0.19        67
weighted avg       0.63      0.46      0.52        67



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
